# Regression and Feature Importance with the Boston Globe

Based on the super-old but super-relevant [Speed Trap: Who Gets a Ticket, Who Gets a Break?](http://archive.boston.com/globe/metro/packages/tickets/).

For data, we'll be using `tickets-warnings.csv`.

If you'd like to kind of ignore everything I say, you can go read [this page](https://investigate.ai/boston-globe-tickets/boston-globe-ticketing-regression/). It assumes some prior knowledge of logistic regression and the `statsmodels` package, though - we're coming into this backwards since we began with sentiment analysis!

In [ ]:
# pip install statsmodels

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 100)

In [3]:
# Unknown data is always coded as 'U', so we'll pass that to na_values
df = pd.read_csv('data/tickets-warnings.csv', na_values='U')

df.head()

,TYPE,CITATION,DATE,DOW,AGENCY,AGENCY2,AGENCY3,LOCAL,OFFICER,LICSTATE,CLASS,CDL,RACE,MINORITY,BLACK,ASIAN,HISPANIC,MIDDLE,NATIVE,SEX,FEMALE,SEARCH,SEARCH2,LOCATE2,LOCATION,TIME,AMPM,TIEMDAY,DAYNIGHT,DAY,DESCRIPT,AMOUNT,MPH,ZONE,MPHOVER,MPHPCT,MPHGROUP,YOB,AGE,AGEGROUP,AGEBAND,ZIP,NEIGHBOR,INNEIGH,REGSTATE,V_MAKE,V_TYPE,V_YEAR,V_AGE,V_AGEGRP,COLOR,HOMESTATE,HOMETOWN,INTOWN,INSTATE,INTOWN2,INSTATE2
0,T,K0001506,20010411,Wednesday,State Police Troop A-4,State Police,S,N,8.247791e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,NaN,NaN,Woburn,Woburn,5,PM,b) afternoon,day,1,SPEEDING,125.0,80,65,15.0,23.0,b) 10 to 15,1980,21.0,21-25,16-25,1876.0,NaN,NaN,MA,NaN,NaN,0,NaN,NaN,NaN,MA,Tewksbury,N,Y,0.0,1
1,T,K0001507,20010417,Tuesday,State Police Troop A-4,State Police,S,N,8.247791e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,F,1.0,NaN,NaN,Somerville,Somerville,10,AM,a) morning,day,1,TRAFFIC VIOLATION,50.0,0,0,NaN,NaN,NaN,1965,36.0,36-40,26-39,2135.0,Allston-Brighton,N,MA,DODG,ARIES,1988,13.0,older,WHITE,MA,Boston,N,Y,0.0,1
2,T,K0001509,20010420,Friday,State Police Troop A-4,State Police,S,N,8.247791e+15,NH,NaN,N,W,W,0.0,0.0,0.0,0.0,0.0,F,1.0,NaN,NaN,Somerville,Somerville,4,PM,b) afternoon,day,1,FAILURE TO STOP,NaN,0,0,NaN,NaN,NaN,1940,61.0,61-65,40+,3062.0,NaN,NaN,NH,NaN,NaN,0,NaN,NaN,NaN,NH,NaN,N,N,0.0,0
3,T,K0001510,20010420,Friday,State Police Troop A-4,State Police,S,N,8.247791e+15,NH,NaN,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,NaN,NaN,Woburn,Woburn,8,PM,c) evening,night,0,KEEP IN RIGHT LANE,100.0,0,0,NaN,NaN,NaN,1949,52.0,51-55,40+,3045.0,NaN,NaN,NH,NaN,NaN,0,NaN,NaN,NaN,NH,NaN,N,N,0.0,0
4,T,K0001511,20010427,Friday,State Police Troop A-4,State Police,S,N,8.247791e+15,MA,D,N,W,W,0.0,0.0,0.0,0.0,0.0,M,0.0,NaN,NaN,Woburn,Woburn,8,AM,a) morning,day,1,SPEEDING,NaN,85,65,20.0,31.0,c) 16 to 20,1977,24.0,21-25,16-25,1835.0,NaN,NaN,MA,MAZD,B3000,2000,1.0,new,BLACK,MA,Haverhill,N,Y,0.0,1


In [4]:
df.TYPE.value_counts()

T    84002
W    82366
Name: TYPE, dtype: int64

## Let's get rid of most of our features

In [5]:
df = df[['TYPE','AGENCY3','SEX','BLACK','ASIAN','HISPANIC','MINORITY','AGE','MPH','MPHOVER','INTOWN', 'INSTATE', 'DAYNIGHT']].copy()
df.head()

,TYPE,AGENCY3,SEX,BLACK,ASIAN,HISPANIC,MINORITY,AGE,MPH,MPHOVER,INTOWN,INSTATE,DAYNIGHT
0,T,S,M,0.0,0.0,0.0,W,21.0,80,15.0,N,Y,day
1,T,S,F,0.0,0.0,0.0,W,36.0,0,NaN,N,Y,day
2,T,S,F,0.0,0.0,0.0,W,61.0,0,NaN,N,N,day
3,T,S,M,0.0,0.0,0.0,W,52.0,0,NaN,N,N,night
4,T,S,M,0.0,0.0,0.0,W,24.0,85,20.0,N,Y,day


## Remove your missing data or sklearn gets angry

In [6]:
print(df.shape)
df = df.dropna().copy()
print(df.shape)

(166368, 13)
(89691, 13)


## Label needs to be 0/1

In [7]:
# TYPE
# T = ticket
# W = warning
df['is_ticketed'] = df.TYPE.replace({
    'T': 1,
    'W': 0
})
df.head()

,TYPE,AGENCY3,SEX,BLACK,ASIAN,HISPANIC,MINORITY,AGE,MPH,MPHOVER,INTOWN,INSTATE,DAYNIGHT,is_ticketed
0,T,S,M,0.0,0.0,0.0,W,21.0,80,15.0,N,Y,day,1
4,T,S,M,0.0,0.0,0.0,W,24.0,85,20.0,N,Y,day,1
5,T,S,M,0.0,0.0,0.0,W,37.0,80,30.0,N,Y,day,1
6,W,S,M,0.0,0.0,0.0,W,30.0,80,15.0,N,N,night,0
7,W,S,F,1.0,0.0,0.0,M,22.0,75,10.0,N,N,night,0


## Setting up our features

In [8]:
df['is_male'] = df.SEX.replace({
    'M': 1,
    'F': 0
    
})
df.head(2)

,TYPE,AGENCY3,SEX,BLACK,ASIAN,HISPANIC,MINORITY,AGE,MPH,MPHOVER,INTOWN,INSTATE,DAYNIGHT,is_ticketed,is_male
0,T,S,M,0.0,0.0,0.0,W,21.0,80,15.0,N,Y,day,1,1
4,T,S,M,0.0,0.0,0.0,W,24.0,85,20.0,N,Y,day,1,1


In [9]:
df['is_white'] = df.MINORITY.replace({
    'W': 1,
    'M': 0
})
df.head(2)

,TYPE,AGENCY3,SEX,BLACK,ASIAN,HISPANIC,MINORITY,AGE,MPH,MPHOVER,INTOWN,INSTATE,DAYNIGHT,is_ticketed,is_male,is_white
0,T,S,M,0.0,0.0,0.0,W,21.0,80,15.0,N,Y,day,1,1,1
4,T,S,M,0.0,0.0,0.0,W,24.0,85,20.0,N,Y,day,1,1,1


In [10]:
df['is_intown'] = df.INTOWN.replace({
    'Y': 1,
    'N': 0
})
df.head(2)

,TYPE,AGENCY3,SEX,BLACK,ASIAN,HISPANIC,MINORITY,AGE,MPH,MPHOVER,INTOWN,INSTATE,DAYNIGHT,is_ticketed,is_male,is_white,is_intown
0,T,S,M,0.0,0.0,0.0,W,21.0,80,15.0,N,Y,day,1,1,1,0
4,T,S,M,0.0,0.0,0.0,W,24.0,85,20.0,N,Y,day,1,1,1,0


In [12]:
feature_columns = ['is_male', 'is_white', 'is_intown']
X = df[feature_columns]
y = df.is_ticketed

In [16]:
y[:3]

0    1
4    1
5    1
Name: is_ticketed, dtype: int64

In [18]:
X.head(3)

,is_male,is_white,is_intown
0,1,1,0
4,1,1,0
5,1,1,0


## Classification

"What is going on in minds of the police??????"

Let's make a model that reproduces what the police do, but with machine learning!

In [19]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=2)

clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [20]:
import eli5

feature_names = list(X.columns)

eli5.show_weights(clf,
                  feature_names=feature_names,
                  target_names=['warning', 'ticket'])

Weight,Feature
0.6431,is_intown
0.1960,is_male
0.1609,is_white


## Let's try another classifier!

In [21]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1e9, solver='lbfgs')

clf.fit(X, y)

LogisticRegression(C=1000000000.0)

In [22]:
import eli5

feature_names = list(X.columns)

eli5.show_weights(clf, feature_names=feature_names)

Weight?,Feature
+0.390,<BIAS>
+0.364,is_male
-0.414,is_white
-0.668,is_intown


## Things that are annoying

Let's make a list.